In [10]:
import pandas as pd
import torch
import sys
import numpy as np

In [11]:
# obtaining data
df = pd.read_csv('data/Advertising.csv').drop('Unnamed: 0', axis=1)

X = torch.Tensor([df.TV.to_numpy(),
                  df.Radio.to_numpy(),
                  df.Newspaper.to_numpy(),
                np.repeat([1], 200)]).transpose(0, 1)
y = torch.Tensor([df.Sales.to_numpy()]).transpose(0, 1)

In [12]:
X.shape

torch.Size([200, 4])

In [13]:
y.shape

torch.Size([200, 1])

In [14]:
# prior parameters
a0 = 1e-2
b0 = 1e-4
c0 = 1e-2
d0 = 1e-4

In [15]:
# pre-process data
N = X.shape[0]
D = 1
if len(X.shape) > 1:
    D = X.shape[1]

X_corr = X.transpose(0, 1).matmul(X)
Xy_corr = X.transpose(0, 1).matmul(y)

an = a0 + N / 2
gammaln_an = torch.lgamma(torch.Tensor([an]))
cn = c0 + D / 2
gammaln_cn = torch.lgamma(torch.Tensor([cn]))

In [16]:
# iterate to find hyperparameters
L_last = -sys.float_info.max
max_iter = 500
E_a = c0 / d0

for i in range(max_iter):
    # covariance and weight of linear model
    invV = E_a * torch.eye(D) + X_corr
    V = torch.inverse(invV)
    logdetV = - torch.logdet(invV)
    w = V.matmul(Xy_corr)

    # parameters of noise model (an remains constant)
    sse = torch.sum((X.matmul(w) - y) ** 2)
    bn = b0 + 0.5 * (sse + E_a * w.transpose(0, 1).matmul(w))
    E_t = an / bn

    # hyperparameters of covariance prior (cn remains constant)
    dn = d0 + 0.5 * (E_t * w.transpose(0, 1).matmul(w) + torch.trace(V))
    E_a = cn / dn

    # variational bound, ignoring constant terms for now
    L = -0.5 * (E_t * sse + torch.sum(torch.sum(X * (X.matmul(V))))) + 0.5 * logdetV \
        - b0 * E_t + gammaln_an - an * torch.log(bn) + an \
        + gammaln_cn - cn * torch.log(dn)
    
    print(L)
    
    # variational bound must grow!
    if L_last > L:
        print('Last bound:', L_last)
        print('Current bound:', L)
        print('Variational bound should not reduce')
        break

    # stop if change in variation bound is < 0.001%
    if abs(L_last - L) < abs(0.00000001 * L):
        break
    L_last = L

    if iter == max_iter:
        print('Bayesian linear regression reached maximum number of iterations')

    # augment variational bound with constant terms
    L = L - 0.5 * (N * torch.log(torch.Tensor([2 * torch.pi])) - D) - torch.lgamma(torch.Tensor([a0])) + a0 * torch.log(torch.Tensor([b0])) \
        - torch.lgamma(torch.Tensor([c0])) + c0 * torch.log(torch.Tensor([d0]))

tensor([[-245.7125]])
tensor([[-235.5653]])
tensor([[-228.0690]])
tensor([[-227.3488]])
tensor([[-227.3345]])
tensor([[-227.3343]])
tensor([[-227.3343]])
tensor([[-227.3343]])


In [17]:
print("InvV:", invV)
print("V:", V)
print("logdetV:", logdetV)
print("w", w)
print("sse:", sse)
print("bn:", bn)
print("E_t:", E_t)
print("dn:", dn)
print("E_a:", E_a)
print("an:", an)
print("gammaln_an:", gammaln_an)
print("cn:", cn)
print("gammaln_cn:", gammaln_cn)

InvV: tensor([[5.7911e+06, 6.9806e+05, 9.1963e+05, 2.9409e+04],
        [6.9806e+05, 1.5211e+05, 1.6495e+05, 4.6528e+03],
        [9.1963e+05, 1.6495e+05, 2.8110e+05, 6.1108e+03],
        [2.9409e+04, 4.6528e+03, 6.1108e+03, 2.0143e+02]])
V: tensor([[ 6.7295e-07, -2.0749e-07, -1.4152e-07, -8.9162e-05],
        [-2.0749e-07,  2.5890e-05, -6.3769e-06, -3.7428e-04],
        [-1.4152e-07, -6.3769e-06,  1.2073e-05, -1.9831e-04],
        [-8.9162e-05, -3.7428e-04, -1.9831e-04,  3.2643e-02]])
logdetV: tensor(-41.5503)
w tensor([[ 4.6140e-02],
        [ 1.9010e-01],
        [-2.0082e-04],
        [ 2.8016e+00]])
sse: tensor(557.3759)
bn: tensor([[284.3359]])
E_t: tensor([[0.3517]])
dn: tensor([[1.4035]])
E_a: tensor([[1.4321]])
an: 100.01
gammaln_an: tensor([359.1802])
cn: 2.01
gammaln_cn: tensor([0.0043])
